In [ ]:
import serial
import time
import tensorflow as tf
import tensorflow.keras as keras
import csv
import numpy as np
from _posture_detection_model import PostureDetectionModel

In [ ]:
# open serial port
ser = serial.Serial('COM5', 9600, timeout=0)

In [ ]:
# Create 200x3 numpy array to store data
ring_buffer = np.zeros((1, 200, 3))

In [ ]:
ring_buffer.shape

In [ ]:
model = PostureDetectionModel(3, 200, 5)
model.build(input_shape=(None, 200, 3))
model.load_weights('posture_detection_model_relu.h5')

In [7]:
ser.flush()


index = 0

while True:
    line = ser.readline()

    data = list(csv.reader([line.decode('utf-8').rstrip()]))
    
     # if ring buffer is full, predict
    if index == 200:
        #predict
        with tf.device('/device:GPU:0'):
            output = model(ring_buffer, training=False)
            output = np.argmax(output, axis=1)
            print(output)
        index = 0
        ser.flush()

    # store data in ring buffer
    ring_buffer[0, index, :] = data[0]
    index += 1

    time.sleep(0.005)